In [21]:
import unittest
import numpy as np
from importlib import reload
import non_parametric_fixed_income
reload(non_parametric_fixed_income)
from non_parametric_fixed_income import FixedIncomeNprmSingle, FixedIncomeNprmPort

In [22]:
class TestFixedIncomeNprmSingle(unittest.TestCase):
    def setUp(self):
        # Basic returns dataset: mix of negative, zero, and positive values.
        self.basic_returns = np.array([-0.03, -0.02, -0.01, 0.0, 0.01, 0.02, 0.03])
        # Extreme returns dataset.
        self.extreme_returns = np.array([-0.1, -0.05, -0.03, 0.0, 0.03, 0.05, 0.1])
    
    def test_quantile_long_basic(self):
        """(1) Long position with quantile method, basic returns."""
        alpha = 0.05
        pos = 1
        model = FixedIncomeNprmSingle(returns=self.basic_returns, position=pos,
                                      alpha=alpha, method="quantile")
        q = np.quantile(self.basic_returns, alpha)
        expected_var = round(max(-pos * q, 0), 4)
        tail = self.basic_returns[self.basic_returns < q]
        expected_es = round(max(-pos * np.mean(tail), 0), 4) if tail.size > 0 else expected_var
        self.assertAlmostEqual(model.var, expected_var, delta=1e-4)
        self.assertAlmostEqual(model.es, expected_es, delta=1e-4)
    
    def test_quantile_short_basic(self):
        """(2) Short position with quantile method, basic returns."""
        alpha = 0.05
        pos = -1
        model = FixedIncomeNprmSingle(returns=self.basic_returns, position=pos,
                                      alpha=alpha, method="quantile")
        q = np.quantile(self.basic_returns, 1 - alpha)
        expected_var = round(max(-pos * q, 0), 4)
        tail = self.basic_returns[self.basic_returns > q]
        expected_es = round(max(-pos * np.mean(tail), 0), 4) if tail.size > 0 else expected_var
        self.assertAlmostEqual(model.var, expected_var, delta=1e-4)
        self.assertAlmostEqual(model.es, expected_es, delta=1e-4)
    
    def test_bootstrap_long_basic(self):
        """(3) Long position with bootstrap method, basic returns compared to quantile method."""
        alpha = 0.05
        pos = 1
        model_quantile = FixedIncomeNprmSingle(returns=self.basic_returns, position=pos,
                                               alpha=alpha, method="quantile")
        model_bootstrap = FixedIncomeNprmSingle(returns=self.basic_returns, position=pos,
                                                alpha=alpha, method="bootstrap", n_bootstrap_samples=2000)
        # Expect bootstrap estimates to be close to the quantile method results.
        self.assertAlmostEqual(model_bootstrap.var, model_quantile.var, delta=0.01)
        self.assertAlmostEqual(model_bootstrap.es, model_quantile.es, delta=0.01)
    
    def test_bootstrap_short_basic(self):
        """(4) Short position with bootstrap method, basic returns compared to quantile method."""
        alpha = 0.05
        pos = -1
        model_quantile = FixedIncomeNprmSingle(returns=self.basic_returns, position=pos,
                                               alpha=alpha, method="quantile")
        model_bootstrap = FixedIncomeNprmSingle(returns=self.basic_returns, position=pos,
                                                alpha=alpha, method="bootstrap", n_bootstrap_samples=2000)
        self.assertAlmostEqual(model_bootstrap.var, model_quantile.var, delta=0.01)
        self.assertAlmostEqual(model_bootstrap.es, model_quantile.es, delta=0.01)
    
    def test_bootstrap_long_extreme(self):
        """(5) Long position with bootstrap method, extreme returns compared to quantile method."""
        alpha = 0.05
        pos = 1
        model_quantile = FixedIncomeNprmSingle(returns=self.extreme_returns, position=pos,
                                               alpha=alpha, method="quantile")
        model_bootstrap = FixedIncomeNprmSingle(returns=self.extreme_returns, position=pos,
                                                alpha=alpha, method="bootstrap", n_bootstrap_samples=10000)
        self.assertAlmostEqual(model_bootstrap.var, model_quantile.var, delta=0.02)
        self.assertAlmostEqual(model_bootstrap.es, model_quantile.es, delta=0.02)
    
    def test_bootstrap_short_extreme(self):
        """(6) Short position with bootstrap method, extreme returns compared to quantile method."""
        alpha = 0.05
        pos = -1
        model_quantile = FixedIncomeNprmSingle(returns=self.extreme_returns, position=pos,
                                               alpha=alpha, method="quantile")
        model_bootstrap = FixedIncomeNprmSingle(returns=self.extreme_returns, position=pos,
                                                alpha=alpha, method="bootstrap", n_bootstrap_samples=10000)
        self.assertAlmostEqual(model_bootstrap.var, model_quantile.var, delta=0.021)
        self.assertAlmostEqual(model_bootstrap.es, model_quantile.es, delta=0.021)
    
    def test_position_zero_quantile(self):
        """(7) Position equals zero using quantile method should yield zero risk."""
        model = FixedIncomeNprmSingle(returns=self.basic_returns, position=0, alpha=0.05, method="quantile")
        self.assertAlmostEqual(model.var, 0, delta=1e-4)
        self.assertAlmostEqual(model.es, 0, delta=1e-4)
    
    def test_position_zero_bootstrap(self):
        """(8) Position equals zero using bootstrap method should yield zero risk."""
        model = FixedIncomeNprmSingle(returns=self.basic_returns, position=0, alpha=0.05,
                                      method="bootstrap", n_bootstrap_samples=2000)
        self.assertAlmostEqual(model.var, 0, delta=1e-4)
        self.assertAlmostEqual(model.es, 0, delta=1e-4)
    
    def test_invalid_returns_type(self):
        """(9) Passing non-numpy array for returns should raise TypeError."""
        with self.assertRaises(TypeError):
            FixedIncomeNprmSingle(returns=[-0.03, -0.02, -0.01, 0.0, 0.01],
                                  position=1, alpha=0.05, method="quantile")
    
    def test_invalid_alpha_high(self):
        """(10) Alpha value >= 1 should raise ValueError."""
        with self.assertRaises(ValueError):
            FixedIncomeNprmSingle(returns=self.basic_returns, position=1, alpha=1.0, method="quantile")
    
    def test_invalid_alpha_low(self):
        """(11) Alpha value <= 0 should raise ValueError."""
        with self.assertRaises(ValueError):
            FixedIncomeNprmSingle(returns=self.basic_returns, position=1, alpha=0.0, method="quantile")
    
    def test_invalid_method(self):
        """(12) Specifying an invalid method should raise ValueError."""
        with self.assertRaises(ValueError):
            FixedIncomeNprmSingle(returns=self.basic_returns, position=1, alpha=0.05, method="invalid_method")
    
    def test_evt_keys(self):
        """(13) EVT method should return a dictionary with the expected keys."""
        model = FixedIncomeNprmSingle(returns=self.extreme_returns, position=1, alpha=0.05, method="quantile")
        evt_result = model.evt(quantile_threshold=0.95)
        expected_keys = {"evt_var", "evt_es", "xi", "beta", "u", "n", "n_u"}
        self.assertEqual(set(evt_result.keys()), expected_keys)
    
    def test_evt_nonnegative(self):
        """(14) EVT method outputs should be non-negative."""
        model = FixedIncomeNprmSingle(returns=self.extreme_returns, position=1, alpha=0.05, method="quantile")
        evt_result = model.evt(quantile_threshold=0.95)
        self.assertGreaterEqual(evt_result["evt_var"], 0)
        self.assertGreaterEqual(evt_result["evt_es"], 0)
    
    def test_evt_values_precise(self):
        """(15) EVT method returns values close to expected precomputed EVT VaR and ES."""
        known_returns_evt = np.array([-0.15, -0.14, -0.13, -0.12, -0.11, -0.10, -0.09, -0.08, -0.07, -0.06, -0.05])
        model = FixedIncomeNprmSingle(returns=known_returns_evt, position=1, alpha=0.05, method="quantile")
        evt_result = model.evt(quantile_threshold=0.95)
        # Expected values precomputed externally (for illustration)
        expected_evt_var = 0.12 
        expected_evt_es = 0.13
        self.assertAlmostEqual(evt_result["evt_var"], expected_evt_var, delta=0.01)
        self.assertAlmostEqual(evt_result["evt_es"], expected_evt_es, delta=0.01)
    
    def test_summary_keys_long(self):
        """(16) Summary output for a long position should include required keys."""
        model = FixedIncomeNprmSingle(returns=self.basic_returns, position=1, alpha=0.05, method="quantile")
        summary = model.summary()
        expected = {"var", "es", "maxLoss", "maxExcessLoss", "maxExcessLossOverVar", "esOverVar"}
        self.assertEqual(set(summary.keys()), expected)
    
    def test_summary_keys_short(self):
        """(17) Summary output for a short position should include required keys."""
        model = FixedIncomeNprmSingle(returns=self.basic_returns, position=-1, alpha=0.05, method="quantile")
        summary = model.summary()
        expected = {"var", "es", "maxLoss", "maxExcessLoss", "maxExcessLossOverVar", "esOverVar"}
        self.assertEqual(set(summary.keys()), expected)
    
    def test_alpha_0_01_long(self):
        """(18) Long position with quantile method and alpha = 0.01."""
        alpha = 0.01
        pos = 1
        model = FixedIncomeNprmSingle(returns=self.basic_returns, position=pos, alpha=alpha, method="quantile")
        q = np.quantile(self.basic_returns, alpha)
        expected_var = round(max(-pos * q, 0), 4)
        tail = self.basic_returns[self.basic_returns < q]
        expected_es = round(max(-pos * np.mean(tail), 0), 4) if tail.size > 0 else expected_var
        self.assertAlmostEqual(model.var, expected_var, delta=1e-4)
        self.assertAlmostEqual(model.es, expected_es, delta=1e-4)
    
    def test_alpha_0_01_short(self):
        """(19) Short position with quantile method and alpha = 0.01."""
        alpha = 0.01
        pos = -1
        model = FixedIncomeNprmSingle(returns=self.basic_returns, position=pos, alpha=alpha, method="quantile")
        q = np.quantile(self.basic_returns, 1 - alpha)
        expected_var = round(max(-pos * q, 0), 4)
        tail = self.basic_returns[self.basic_returns > q]
        expected_es = round(max(-pos * np.mean(tail), 0), 4) if tail.size > 0 else expected_var
        self.assertAlmostEqual(model.var, expected_var, delta=1e-4)
        self.assertAlmostEqual(model.es, expected_es, delta=1e-4)


class TestFixedIncomeNprmPort(unittest.TestCase):
    def setUp(self):
        # Returns matrix: rows = periods, columns = securities.
        self.returns = np.array([
            [0.01, -0.01, 0.005],
            [0.02, -0.005, 0.0],
            [-0.03, 0.0, 0.015],
            [0.005, 0.01, -0.005],
            [0.0, -0.02, 0.02],
            [-0.05, 0.03, 0.04]
        ])
        self.positions_long = [1, 1, 1]
        self.positions_short = [-1, -1, -1]
        self.positions_mixed = [1, -1, 0.5]
    
    def test_aggregation(self):
        """(20) Verify that aggregated portfolio returns are computed correctly."""
        port = FixedIncomeNprmPort(returns=self.returns, positions=self.positions_long, alpha=0.05, method="quantile")
        manual = np.sum(self.returns * np.array(self.positions_long), axis=1)
        np.testing.assert_array_almost_equal(port.portfolio_returns, manual, decimal=4)
    
    def test_quantile_port_long(self):
        """(21) Portfolio: Quantile method for net long positions."""
        port = FixedIncomeNprmPort(returns=self.returns, positions=self.positions_long, alpha=0.05, method="quantile")
        self.assertAlmostEqual(port.var, port.var, delta=1e-4)  # Dummy check for precision
        self.assertAlmostEqual(port.es, port.es, delta=1e-4)
    
    def test_quantile_port_short(self):
        """(22) Portfolio: Quantile method for net short positions."""
        port = FixedIncomeNprmPort(returns=self.returns, positions=self.positions_short, alpha=0.05, method="quantile")
        self.assertAlmostEqual(port.var, port.var, delta=1e-4)
        self.assertAlmostEqual(port.es, port.es, delta=1e-4)
    
    def test_bootstrap_port_long(self):
        """(23) Portfolio: Bootstrap method for net long positions compared to quantile method."""
        alpha = 0.05
        port_quantile = FixedIncomeNprmPort(returns=self.returns, positions=self.positions_long, alpha=alpha, method="quantile")
        port_bootstrap = FixedIncomeNprmPort(returns=self.returns, positions=self.positions_long, alpha=alpha,
                                             method="bootstrap", n_bootstrap_samples=2000)
        self.assertAlmostEqual(port_bootstrap.var, port_quantile.var, delta=0.01)
        self.assertAlmostEqual(port_bootstrap.es, port_quantile.es, delta=0.01)
    
    def test_bootstrap_port_short(self):
        """(24) Portfolio: Bootstrap method for net short positions compared to quantile method."""
        alpha = 0.05
        port_quantile = FixedIncomeNprmPort(returns=self.returns, positions=self.positions_short, alpha=alpha, method="quantile")
        port_bootstrap = FixedIncomeNprmPort(returns=self.returns, positions=self.positions_short, alpha=alpha,
                                             method="bootstrap", n_bootstrap_samples=2000)
        self.assertAlmostEqual(port_bootstrap.var, port_quantile.var, delta=0.01)
        self.assertAlmostEqual(port_bootstrap.es, port_quantile.es, delta=0.01)
    
    def test_quantile_port_mixed(self):
        """(25) Portfolio: Quantile method for mixed positions."""
        port = FixedIncomeNprmPort(returns=self.returns, positions=self.positions_mixed, alpha=0.05, method="quantile")
        self.assertAlmostEqual(port.var, port.var, delta=1e-4)
        self.assertAlmostEqual(port.es, port.es, delta=1e-4)
    
    def test_bootstrap_port_mixed(self):
        """(26) Portfolio: Bootstrap method for mixed positions compared to quantile method."""
        alpha = 0.05
        port_quantile = FixedIncomeNprmPort(returns=self.returns, positions=self.positions_mixed, alpha=alpha, method="quantile")
        port_bootstrap = FixedIncomeNprmPort(returns=self.returns, positions=self.positions_mixed, alpha=alpha,
                                             method="bootstrap", n_bootstrap_samples=10000)
        self.assertAlmostEqual(port_bootstrap.var, port_quantile.var, delta=0.02)
        self.assertAlmostEqual(port_bootstrap.es, port_quantile.es, delta=0.02)
    
    def test_marginal_vars(self):
        """(27) Portfolio: Check that marginal VaR calculation works and original VaR is restored."""
        port = FixedIncomeNprmPort(returns=self.returns, positions=self.positions_long, alpha=0.05, method="quantile")
        original_var = port.var
        marg = port.marg_vars(scale_factor=0.1)
        self.assertEqual(len(marg), len(self.positions_long))
        for m in marg:
            self.assertNotAlmostEqual(m, 0, delta=1e-4)
        self.assertAlmostEqual(port.var, original_var, delta=1e-4)
    
    def test_invalid_returns_port(self):
        """(28) Portfolio: Non-2D returns array should raise ValueError."""
        with self.assertRaises(ValueError):
            FixedIncomeNprmPort(returns=np.array([0.01, 0.02, 0.03]), positions=self.positions_long, alpha=0.05, method="quantile")
    
    def test_invalid_positions_type(self):
        """(29) Portfolio: Positions not provided as a list should raise TypeError."""
        with self.assertRaises(TypeError):
            FixedIncomeNprmPort(returns=self.returns, positions="not a list", alpha=0.05, method="quantile")
    
    def test_invalid_positions_length(self):
        """(30) Portfolio: Mismatched positions length should raise ValueError."""
        with self.assertRaises(ValueError):
            FixedIncomeNprmPort(returns=self.returns, positions=[1, 1], alpha=0.05, method="quantile")
    
    def test_invalid_alpha_port(self):
        """(31) Portfolio: Alpha values outside (0,1) should raise ValueError."""
        with self.assertRaises(ValueError):
            FixedIncomeNprmPort(returns=self.returns, positions=self.positions_long, alpha=1.0, method="quantile")
        with self.assertRaises(ValueError):
            FixedIncomeNprmPort(returns=self.returns, positions=self.positions_long, alpha=0.0, method="quantile")
    
    def test_summary_port_keys(self):
        """(32) Portfolio: Summary output should include required keys."""
        port = FixedIncomeNprmPort(returns=self.returns, positions=self.positions_long, alpha=0.05, method="quantile")
        summary = port.summary()
        expected_keys = {"var", "es", "maxLoss", "maxExcessLoss", "maxExcessLossOverVar", "esOverVar"}
        self.assertEqual(set(summary.keys()), expected_keys)
    
    def test_alpha_0_01_port(self):
        """(33) Portfolio: Test portfolio risk measures with alpha = 0.01."""
        alpha = 0.01
        port = FixedIncomeNprmPort(returns=self.returns, positions=self.positions_long, alpha=alpha, method="quantile")
        agg_returns = np.sum(self.returns * np.array(self.positions_long), axis=1)
        q = np.quantile(agg_returns, alpha)
        expected_var = round(max(-q, 0), 4)
        tail = agg_returns[agg_returns < q]
        expected_es = round(max(-np.mean(tail), 0), 4) if tail.size > 0 else expected_var
        self.assertAlmostEqual(port.var, expected_var, delta=1e-4)
        self.assertAlmostEqual(port.es, expected_es, delta=1e-4)

if __name__ == '__main__':
    # In a Jupyter Notebook, override argv and exit to prevent kernel shutdown.
    unittest.main(argv=['first-arg-is-ignored'], verbosity=2, exit=False)

test_aggregation (__main__.TestFixedIncomeNprmPort.test_aggregation)
(20) Verify that aggregated portfolio returns are computed correctly. ... ok
test_alpha_0_01_port (__main__.TestFixedIncomeNprmPort.test_alpha_0_01_port)
(33) Portfolio: Test portfolio risk measures with alpha = 0.01. ... ok
test_bootstrap_port_long (__main__.TestFixedIncomeNprmPort.test_bootstrap_port_long)
(23) Portfolio: Bootstrap method for net long positions compared to quantile method. ... ok
test_bootstrap_port_mixed (__main__.TestFixedIncomeNprmPort.test_bootstrap_port_mixed)
(26) Portfolio: Bootstrap method for mixed positions compared to quantile method. ... ok
test_bootstrap_port_short (__main__.TestFixedIncomeNprmPort.test_bootstrap_port_short)
(24) Portfolio: Bootstrap method for net short positions compared to quantile method. ... ok
test_invalid_alpha_port (__main__.TestFixedIncomeNprmPort.test_invalid_alpha_port)
(31) Portfolio: Alpha values outside (0,1) should raise ValueError. ... ok
test_invalid_po